In [1]:
# Here, we show how a standard adversarial attack evaluation can be integrated into an end-to-end Flow pipeline. The example does not produce interesting results as it is run on a pretrained DistilBERT model for demo purposes.

In [2]:
# Load dummy train and test data from package
import pkg_resources
TRAIN_PATH = pkg_resources.resource_filename(
            'kitanaqa', 'support/unittest-squad.json')
EVAL_PATH = pkg_resources.resource_filename(
            'kitanaqa', 'support/unittest-squad.json')

In [3]:
from transformers import HfArgumentParser, TrainingArguments
from kitanaqa.trainer.arguments import ModelArguments
from kitanaqa.trainer.utils import load_and_cache_examples, build_flow
from kitanaqa.trainer.run_pipeline import _setup
hparams = {
            "model_type" : "distilbert",
            "model_name_or_path" : "distilbert-base-uncased",
            "output_dir" : "./example_outputs",
            "cache_dir" : "./example_outputs",
            "data_dir" : "./example_outputs",
            "train_file_path" : TRAIN_PATH,
            "predict_file_path" : {"dev-v1.1": EVAL_PATH},
            "aug_file_path" : None,
            "do_aug" : False,
            "do_alum" : False,
            "alpha" : 5,
            "eps" : 1e-4,
            "eta" : 1e-5,
            "sigma" : 1e-3,
            "do_train" : False,
            "do_adv_eval" : True,  # Flag for adversarial attack instead of normal evaluation
            "do_eval" : True,
            "per_device_train_batch_size" : 2,
            "per_device_eval_batch_size" : 1,
            "gradient_accumulation_steps" : 2,
            "eval_all_checkpoints" : False,
            "num_train_epochs" : 1,
            "max_steps" : -1,
            "save_steps" : 1,
            "seed" : 512,
            "fp16" : False
}
# Initialize args
parser = HfArgumentParser(dataclass_types=[ModelArguments, TrainingArguments])
model_args, training_args = parser.parse_dict(hparams)
model, tokenizer, train_dataset = _setup(model_args, training_args)

11/12/2020 17:46:31 - kitanaqa - WARNING - Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
11/12/2020 17:46:31 - WARNING - kitanaqa -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
11/12/2020 17:46:31 - kitanaqa - INFO - Training/evaluation parameters TrainingArguments(output_dir='./example_outputs', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluate_during_training=False, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=2, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov12_17-46-31_Aarons-MBP-7.attlocal.net', logging_first_step=False, logging_steps=500, save_steps=1, save_total_limit=None, no_cuda=F

In [4]:
# Construct a Flow pipeline consisting of a parent step (training) conditionally followed by a child step (evaluation)
f = build_flow(
            (model_args, training_args),
            model=model,
            tokenizer=tokenizer,
            train_dataset=train_dataset)
if f:
    f.run()

[2020-11-13 01:46:44] INFO - prefect.FlowRunner | Beginning Flow run for 'default'


11/12/2020 17:46:44 - INFO - prefect.FlowRunner -   Beginning Flow run for 'default'


[2020-11-13 01:46:44] INFO - prefect.TaskRunner | Task 'eval': Starting task run...


11/12/2020 17:46:44 - INFO - prefect.TaskRunner -   Task 'eval': Starting task run...
11/12/2020 17:46:44 - kitanaqa.trainer.utils - WARNING - You are running a non-local model checkpoint. This may or may not be what you intended.
11/12/2020 17:46:44 - WARNING - kitanaqa.trainer.utils -   You are running a non-local model checkpoint. This may or may not be what you intended.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the check

===alum_results:  OrderedDict([('exact', 0.0), ('f1', 16.338624338624335), ('total', 10), ('HasAns_exact', 0.0), ('HasAns_f1', 16.338624338624335), ('HasAns_total', 10), ('best_exact', 0.0), ('best_exact_thresh', 0.0), ('best_f1', 16.338624338624335), ('best_f1_thresh', 0.0)])
[2020-11-13 01:47:10] INFO - prefect.TaskRunner | Task 'eval': finished task run for task with final state: 'Success'


11/12/2020 17:47:10 - INFO - prefect.TaskRunner -   Task 'eval': finished task run for task with final state: 'Success'


[2020-11-13 01:47:10] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


11/12/2020 17:47:10 - INFO - prefect.FlowRunner -   Flow run SUCCESS: all reference tasks succeeded


In [ ]:
f.result